In [ ]:
# Bloco 1: Upload dos Dados
from google.colab import files
import json

# Carregar o arquivo de dataset
uploaded = files.upload()

# Carregar o dataset
dataset_file = next(iter(uploaded.keys()))  # Pega o nome do primeiro arquivo enviado
with open(dataset_file, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Exibir o primeiro item do dataset carregado
print(f"Primeiro item do dataset carregado: {dataset[0]}")



Saving dataset.json to dataset.json
Primeiro item do dataset carregado: {'lyric': 'Me chamaste para caminhar Na vida contigo Decidi para sempre seguir-te Não voltar atrás Me puseste uma brasa no peito Uma flecha na alma É difícil agora viver Sem lembrar-me de ti. Te amarei Senhor, Te amarei SenhorEu só encontro a paz e a alegria Bem perto de ti, te amarei Te amarei Senhor, Te amarei Senhor Eu só encontro a paz e a alegria Bem perto de ti. Eu pensei muitas vezes calar E não dar nem resposta. Eu pensei na fuga esconder-me Ir longe de ti. Mas tua força venceu E ao final eu fiquei seduzido E é difícil agora viver Sem saudades de ti. Te amarei Senhor, Te amarei Senhor Eu só encontro a paz e a alegria Bem perto de ti, te amarei Te amarei Senhor, Te amarei Senhor Eu só encontro a paz e a alegria Bem perto de ti. Ó Jesus não me deixe jamais Caminhar solitário. Pois conheces A minha fraqueza e o meu coração Vem, ensina-me a viver A vida na tua presençaNo amor dos irmãosNa alegria, na paz, na un

In [ ]:
# Bloco 2: Processamento dos Dados
from sklearn.preprocessing import MultiLabelBinarizer

# Extrair tópicos únicos, ignorando tópicos vazios
all_topics = sorted(set(topic.strip() for item in dataset for topic in item['keys'] if topic.strip()))

# Inicializar o binarizador para transformar os tópicos em vetores binários
mlb = MultiLabelBinarizer(classes=all_topics)
y = mlb.fit_transform([item['keys'] for item in dataset])

# Exibir uma amostra dos tópicos processados
print(f"Tópicos únicos: {all_topics}")
print(f"Primeiro item binarizado: {y[0]}")




Tópicos únicos: ['Abismo', 'Adoração', 'Alegria', 'Aliança', 'Altar', 'Amor', 'Amém', 'Animais', 'Anjo', 'Arrependimento', 'Autoridade', 'Banquete messiânico', 'Batismo', 'Belém', 'Benção', 'Bondade', 'Caminho', 'Caridade', 'Comunhão', 'Comunidade', 'Concórdia', 'Confissão', 'Conhecimento de Deus', 'Consagração', 'Conversão', 'Coração', 'Cordeiro', 'Corpo', 'Criação', 'Cristo', 'Cristão', 'Cruz', 'Curas', 'Céu', 'Deserto', 'Deus', 'Dia do senhor', 'Discernimento', 'Escravidão', 'Esperança', 'Espírito Santo', 'Eucaristia', 'Evangelho', 'Família', 'Fidelidade', 'Filho de Deus', 'Fogo', 'Fé', 'Glória', 'Gratidão', 'Graça', 'Humilde', 'Igreja', 'Irmãos', 'Jesus', 'Justo', 'Liberdade', 'Libertação', 'Louvor a Deus', 'Luz', 'Mandamentos', 'Maria', 'Messias', 'Milagre', 'Misericórdia', 'Missão', 'Mistério', 'Morte', 'Mulher', 'Oração', 'Pai', 'Palavra de Deus', 'Pastor', 'Paz', 'Perdão', 'Peregrinação', 'Perseguição', 'Perseverança', 'Pobreza', 'Primogênito', 'Promessa', 'Providência', 'Purez

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


In [ ]:
# Bloco 3: Definição de Parâmetros e Preparação para o Treinamento
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# Definir o nome do modelo pré-treinado BERT
model_name = 'bert-base-uncased'

# Carregar o tokenizer BERT
tokenizer = BertTokenizer.from_pretrained(model_name)

# Definir uma classe Dataset personalizada para uso no treinamento do modelo
class CantosDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        item = {key: val.squeeze() for key, val in inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

# Preparar os dados de entrada e criar o dataset
texts = [item['lyric'] for item in dataset]
dataset_object = CantosDataset(texts, y, tokenizer)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Bloco 4: Divisão do conjunto de dados e Definição do Trainer
from sklearn.model_selection import train_test_split

# Divisão do conjunto de dados em treinamento e avaliação
train_texts, eval_texts, train_labels, eval_labels = train_test_split(texts, y, test_size=0.2, random_state=42)

# Criar os datasets de treinamento e avaliação
train_dataset = CantosDataset(train_texts, train_labels, tokenizer)
eval_dataset = CantosDataset(eval_texts, eval_labels, tokenizer)

# Definição dos parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=150,
    save_total_limit=3,
    load_best_model_at_end=True
)

# Inicializando o modelo BERT para classificação
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(all_topics))

# Inicializando o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=None,  # Pode ser uma função que você definir, se desejar
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()
model.save_pretrained('./modelo_finetuned')
tokenizer.save_pretrained('./modelo_finetuned')

Epoch,Training Loss,Validation Loss
1,No log,0.593257
2,No log,0.528518
3,No log,0.475880
4,No log,0.434276
5,No log,0.397329
6,No log,0.366624
7,No log,0.340866
8,No log,0.319556
9,No log,0.302093
10,No log,0.287951


('./modelo_finetuned/tokenizer_config.json',
 './modelo_finetuned/special_tokens_map.json',
 './modelo_finetuned/vocab.txt',
 './modelo_finetuned/added_tokens.json')

In [ ]:
# Bloco 6: Validação e Avaliação do Modelo
import numpy as np
from sklearn.metrics import classification_report
from transformers import BertForSequenceClassification, BertTokenizer

# Carregar o modelo e o tokenizer ajustados
model = BertForSequenceClassification.from_pretrained('./modelo_finetuned')
tokenizer = BertTokenizer.from_pretrained('./modelo_finetuned')

# Função para avaliação do modelo
def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions >= 0.5).astype(int)  # Converter para 0 ou 1 com limiar de 0.5
    report = classification_report(labels, preds, target_names=mlb.classes_, output_dict=True)
    return report

# Usar o método predict para obter as previsões e os rótulos
predictions = trainer.predict(eval_dataset)

# Obter os resultados de classificação
report = compute_metrics(predictions)

# Calcular a acurácia
accuracy = np.sum(predictions.predictions.argmax(axis=1) == predictions.label_ids.argmax(axis=1)) / len(predictions.label_ids)

# Exibir o relatório em formato desejado
print(f"\nModelo: {model_name} ({training_args.num_train_epochs} épocas)\n")
print("\t\tprecision    recall  f1-score   support\n")
for topic in all_topics:
    if topic in report:
        print(f"{topic:<12} {report[topic]['precision']:.2f}      {report[topic]['recall']:.2f}      {report[topic]['f1-score']:.2f}         {report[topic]['support']}")

# Exibir métricas globais
print(f"\naccuracy                           {accuracy:.2f}        {len(predictions.label_ids)}")
print(f"macro avg       {report['macro avg']['precision']:.2f}      {report['macro avg']['recall']:.2f}      {report['macro avg']['f1-score']:.2f}        {report['macro avg']['support']}")
print(f"weighted avg       {report['weighted avg']['precision']:.2f}      {report['weighted avg']['recall']:.2f}      {report['weighted avg']['f1-score']:.2f}        {report['weighted avg']['support']}")



Modelo: bert-base-uncased (150 épocas)

		precision    recall  f1-score   support

Abismo       0.00      0.00      0.00         1.0
Adoração     0.00      0.00      0.00         6.0
Alegria      0.67      0.20      0.31         10.0
Aliança      0.00      0.00      0.00         1.0
Altar        0.00      0.00      0.00         0.0
Amor         0.60      0.46      0.52         13.0
Amém         0.00      0.00      0.00         0.0
Animais      0.00      0.00      0.00         0.0
Anjo         0.00      0.00      0.00         1.0
Arrependimento 0.00      0.00      0.00         0.0
Autoridade   0.00      0.00      0.00         0.0
Banquete messiânico 0.00      0.00      0.00         0.0
Batismo      0.00      0.00      0.00         1.0
Belém        0.00      0.00      0.00         1.0
Benção       0.00      0.00      0.00         1.0
Bondade      0.00      0.00      0.00         0.0
Caminho      0.00      0.00      0.00         6.0
Caridade     0.00      0.00      0.00         0.0
Comun

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/me

In [ ]:
!zip -r modelo_finetuned.zip ./modelo_finetuned

  adding: modelo_finetuned/ (stored 0%)
  adding: modelo_finetuned/special_tokens_map.json (deflated 42%)
  adding: modelo_finetuned/config.json (deflated 73%)
  adding: modelo_finetuned/model.safetensors (deflated 7%)
  adding: modelo_finetuned/tokenizer_config.json (deflated 75%)
  adding: modelo_finetuned/vocab.txt (deflated 53%)


In [ ]:
# Carregar as bibliotecas necessárias
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import json

# Carregar o modelo e o tokenizer fine-tuned
model = BertForSequenceClassification.from_pretrained('./modelo_finetuned')
tokenizer = BertTokenizer.from_pretrained('./modelo_finetuned')

# Função para fazer previsões com o modelo treinado
def predict_topics(model, tokenizer, user_input):
    # Tokenizar a entrada do usuário
    inputs = tokenizer(user_input, max_length=512, truncation=True, padding='max_length', return_tensors='pt')

    # Mover os tensores para o mesmo dispositivo que o modelo (GPU ou CPU)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    # Colocar o modelo em modo de avaliação e fazer a previsão
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar a função sigmoide para obter as previsões em forma de probabilidades
    probabilities = torch.sigmoid(logits)

    # Converter as probabilidades em rótulos binários (0 ou 1) usando um limiar de 0.5
    preds = (probabilities >= 0.5).int().cpu().numpy()

    # Obter os tópicos previstos usando o MultiLabelBinarizer
    predicted_topics = mlb.inverse_transform(preds)

    return predicted_topics[0]  # Retornar a primeira (e única) previsão

# Solicitar entrada do usuário
user_input = input("Digite a letra do canto: ")

# Chamar a função de previsão
found_topics = predict_topics(model, tokenizer, user_input)

# Imprimir os tópicos encontrados
print("Tópicos encontrados pelo modelo:")
if found_topics:
    for topic in found_topics:
        print(f"- {topic}")
else:
    print("Nenhum tópico encontrado.")
